
# Master Informatique, parcours Data Mining

### Carnets de note Python pour le cours de Text Mining

Julien Velcin, laboratoire ERIC, Université Lyon 2

## Représentation des documents (partie 2)

Différentes solutions existent pour représenter un document dans un espace vectoriel :

* espace des mots (avec différents types de pondération : TF, TFxIDF, OKAPI)
* espace sémantique de faible dimension :
  * **approche de plongement (*sentence/document embedding*)**
  * approche thématique

On va déployer deux méthodes simples basées sur des plongements qui ne prennent pas en compte l'ordre des mots :
- solution naïve : centre d'inertie des vecteurs codant le sens des mots pris indépendamment
- méthode Doc2Vec

D'autres méthodes ont été proposés récemment mais elles se basent souvent sur des approches plus complexes voire coûteuses (comme le Transformer). Des éléments sont données à la fin de ce carnet.

Avant de voir comment construire ces représentations, voyons comment utiliser des plongements de mots pré-appris avec la librairie *spacy*.

In [76]:
import numpy as np

In [77]:
import spacy

# Il faut avoir installé la ressource en local en version "large" (d'où le "lg" à la fin) si on veut avoir accès aux vecteurs : 
#python -m spacy download fr_core_news_lg

nlp = spacy.load('fr_core_news_lg')
#nlp = spacy.load('en_core_web_sm')

La première action que nous pouvons faire est d'interroger la librairie sur les mots les plus proches d'une requête :

In [78]:
def find_close_words(word):
    sim_words = nlp.vocab.vectors.most_similar(
        np.asarray([nlp.vocab.vectors[nlp.vocab.strings[word]]]), n=10)
    return [nlp.vocab.strings[w] for w in sim_words[0][0]]

find_close_words("roi")

# attention, nlp.vocab.strings fonctionne dans les deux sens :
#  - retourne l'identifiant (unique) correspondant à un mot
#  - retourne le mot correspondant à un identifiant

['roi',
 'Roi',
 'prince',
 'monarque',
 'empereur',
 'régent',
 'Empereur',
 'suzerain',
 'coempereur',
 'l\x92empereur']

On peut également calculer des similarités entre mots.

In [79]:
print("entre roi et reine : {}".format(nlp("roi").similarity(nlp("reine"))))
print("entre roi et trône : {}".format(nlp("roi").similarity(nlp("trône"))))
print("entre roi et oiseau : {}".format(nlp("roi").similarity(nlp("oiseau"))))

entre roi et reine : 0.628108097894871
entre roi et trône : 0.6489303722402299
entre roi et oiseau : 0.10113929594015392


Les vecteurs qui représentent les mots sont directement accessibles si besoin.

In [80]:
nlp("roi").vector

array([ 5.6351e+00, -4.7504e+00,  2.9866e+00, -6.5048e-02,  5.6314e+00,
       -2.6488e+00,  1.6728e+00,  3.6799e+00,  2.2033e+00, -2.3401e-01,
        1.7170e+00, -3.0931e+00,  3.8267e+00,  1.8726e-01, -2.2761e+00,
       -2.1582e+00,  3.8331e+00, -1.1703e-01,  1.2575e+00,  3.2968e+00,
       -1.1991e+00,  5.0724e-01, -2.6204e+00,  2.1288e+00,  9.9307e-01,
       -3.1832e+00, -2.3930e+00,  1.3869e+00, -1.7312e+00,  4.5163e+00,
        9.9608e-01, -3.1276e+00, -1.5539e+00,  1.0525e+00, -5.3267e-01,
       -5.5565e+00,  8.4429e-01,  4.9819e+00, -4.6006e-01,  2.2781e+00,
       -1.0639e+00,  1.2235e+00, -4.5606e+00,  3.4419e-01,  1.9167e+00,
       -1.5078e+00,  3.6443e+00,  2.8392e+00,  4.9837e-01, -5.5096e-02,
       -7.9940e+00, -1.9028e+00, -3.9536e+00,  3.7232e-03,  2.3186e+00,
        5.6297e+00,  2.2466e+00,  5.2483e-01,  1.2710e+00, -5.8875e-01,
       -2.2971e+00,  1.4909e-01,  2.7780e-03,  3.9281e+00,  6.5166e-01,
        8.9464e-02,  3.2467e+00,  9.5456e-01, -2.4231e+00, -4.38

On peut même résoudre des problèmes d'analogie. Par ex., qu'est-ce qui est à "femme" ce que "homme" est à "roi" ? Ou la relation "capitale de".

In [81]:
def close_words_from_vector(vec):
    ms = nlp.vocab.vectors.most_similar(np.array([vec]), n=10)
    return [nlp.vocab.strings[w] for w in ms[0][0]]

In [82]:
analogie = nlp("roi").vector-nlp("homme").vector+nlp("femme").vector
close_words_from_vector(analogie)

['roi',
 'reine',
 'Roi',
 'prince',
 'régent',
 'duc',
 'princesse',
 'monarque',
 'suzerain',
 'coempereur']

In [83]:
analogie = nlp("France").vector-nlp("Paris").vector+nlp("Berlin").vector
close_words_from_vector(analogie)

['Allemagne',
 'l´Allemagne',
 'ex-Allemagne',
 'lAllemagne',
 'Allemagne-',
 'Europe',
 'Grande-Bretagne',
 'ouest-allemande',
 'est-allemande',
 'Allemagnes']

## Approche naïve

Nous allons calculer des représentations vectorielles de documents comme le centre d'inertie des mots qui le composent.

In [84]:
import os

with open(os.path.join("datasets", "Frank Herbert - Dune.txt")) as f:
    lines = [line.strip() for line in f.readlines()]

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(stop_words="english", max_df=0.5, min_df=3, max_features=1000)
tf_vectorizer.fit(lines)
D = tf_vectorizer.transform(lines)

features = tf_vectorizer.get_feature_names()

/Users/jvelcin/arm/envs/cours21_spacy/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


On récupère la taille des plongements car elle constitue la dimension de l'espace dans lequel on va plonger les documents.

In [86]:
dim = len(nlp.vocab.vectors[nlp.vocab.strings["dune"]])

In [111]:
ndocs, nwords = D.shape
print(ndocs)

8608


On définit une fonction qui calcule le centre d'inertie d'un ensemble de vecteurs mots.

In [88]:
import numpy as np

def centre(d):
    m = np.zeros(shape=(1,dim))
    nbw = 0
    for w in d:
        try:
            v = nlp.vocab.vectors[nlp.vocab.strings[str(w)]]
            m = np.append(m, v.reshape((1,dim)), axis=0)     
            nbw += 1
        except:
            pass
    seuil = True
    if nbw>0:
        return (nbw, np.sum(m, axis=0)/nbw) # la normalisation est inutile si on utilise le cosinus
    else:
        return (0, m)

Puis on calcule la représentation pour chaque document du corpus. On en profite pour sauvegarder une liste avec la taille des documents (ici, le nombre de mots ayant un vecteur associé dans le plongement).

In [89]:
nbw_docs = []
i = 0
doc_vec = np.zeros(shape=(ndocs,dim))
id_docs_nonvides = []
for d in tf_vectorizer.inverse_transform(D):
    nbw, r = centre(d)
    doc_vec[i] = r.reshape((1,dim))
    nbw_docs.append(nbw)
    i += 1

In [90]:
#len(doc_vec[0])
doc_vec[0:4]

array([[ 2.09649992, -3.93840003, -1.81669998, ..., -0.72492999,
         2.34240007,  0.1106    ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [91]:
nbw_docs[10:20]

[20, 1, 0, 6, 8, 11, 22, 8, 6, 4]

Il ne reste plus qu'à sauvegarder l'information pour une utilisation future.

In [92]:
col_p = np.array(nbw_docs).reshape(ndocs,1)
col_ids = np.arange(1, ndocs+1).reshape(ndocs,1)
data_to_save = np.hstack([doc_vec, col_p, col_ids])
np.savetxt('vec_doc_naive.csv', data_to_save, delimiter='\t')

## Dov2Vec

Doc2Vec est une extension des approches Word2Vec dans lesquelles on ajoute un "token" associé à chaque document (ici, un paragraphe). Il existe deux versions de cet algorithme (Le and Mikolov, 2014) :

* PV-DM : Distributed Memory Models of Paragraph Vectors
* PV-DBOW : Distributed Bag of Words version of Paragraph Vector

<table style="border:0;">
<tr style="border:0;">
    <td><img src="img/PVDM.png" style='height: 200px'/></td>
    <td><img src="img/PVDOBW.png" style='height: 200px'/></td>
    </tr>
    </table>

Il faut formatter les données pour pouvoir les données en entrée de l'algorithme Doc2Vec.

In [93]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# on rajoute une taille minimale dès à présent
min_docs = 4

tagged_docs = []
nbw_docs = []
for i, list_tokens in enumerate(tf_vectorizer.inverse_transform(D)):
    nbw = len(list_tokens)
    nbw_docs.append(nbw)
    if nbw > min_docs:        
        tagged_docs.append(TaggedDocument(words=list_tokens, tags=[str(i+1)]))

In [94]:
len(tagged_docs)
tagged_docs[0:4]

[TaggedDocument(words=array(['arrakis', 'begin', 'beginning', 'bene', 'born', 'caladan', 'care',
        'dib', 'dune', 'emperor', 'fact', 'gesserit', 'known', 'knows',
        'life', 'lived', 'muad', 'padishah', 'place', 'planet', 'special',
        'study', 'taking', 'time', 'year', 'years'], dtype='<U13'), tags=['11']),
 TaggedDocument(words=array(['arrakis', 'boy', 'came', 'mother', 'old', 'paul', 'reached'],
       dtype='<U13'), tags=['14']),
 TaggedDocument(words=array(['ancient', 'atreides', 'caladan', 'change', 'family', 'feeling',
        'home', 'night', 'stone', 'weather'], dtype='<U13'), tags=['15']),
 TaggedDocument(words=array(['allowed', 'bed', 'door', 'lay', 'let', 'moment', 'old', 'passage',
        'paul', 'room', 'woman'], dtype='<U13'), tags=['16'])]

In [95]:
dim_d2v = 10

#model_doc2vec = Doc2Vec(tagged_docs, vector_size=dim_d2v, window = 3, iter = 1000)
model_doc2vec = Doc2Vec(tagged_docs, vector_size=dim_d2v, window = 3)
model_doc2vec.train(tagged_docs, total_examples = len(tagged_docs), epochs = 1000)


In [96]:
set1 = set(features)
set2 = set(model_doc2vec.wv.index_to_key)
set1.difference(set2)

{'10', 'soo'}

In [99]:
model_doc2vec.dv

In [100]:
from nltk.tokenize import word_tokenize

test_doc = word_tokenize("Dune, the spice planet".lower())
test_doc_vector = model_doc2vec.infer_vector(test_doc)
res = model_doc2vec.dv.most_similar(positive = [test_doc_vector])
print(res)

[('8355', 0.9608270525932312), ('7379', 0.9539141654968262), ('7145', 0.9419770836830139), ('1817', 0.9397681355476379), ('8490', 0.9245098829269409), ('7593', 0.9177431464195251), ('7577', 0.9100067019462585), ('6730', 0.9078817367553711), ('7412', 0.9061649441719055), ('6836', 0.8994591236114502)]


In [101]:
for i, s in res:
    ind_doc = int(i)
    print("%s (%s): %s" % (i, s, lines[ind_doc-1]))

8355 (0.9608270525932312): DUNE MEN: idiomatic for open sand workers, spice hunters and the like on Arrakis. Sandworkers. Spiceworkers.
7379 (0.9539141654968262): Paul looked at her. "For the Guild's permission to land. The Guild will strand on Arrakis any force that lands without permission."
7145 (0.9419770836830139): "The evidence is not here," Paul said. "It's in Tabr sietch, far to the south, but if --"
1817 (0.9397681355476379): "Not from the deep desert," Kynes said. "Men have walked out of the second zone several times. They've survived by crossing the rock areas where worms seldom go."
8490 (0.9245098829269409): POLING THE SAND: the art of placing plastic and fiber poles in the open desert wastes of Arrakis and reading the patterns etched on the poles by sandstorms as a clue to weather prediction.
7593 (0.9177431464195251): "His people scream his name as they leap into battle. The women throw their babies at us and hurl themselves onto our knives to open a wedge for their men 

In [102]:
print(len(model_doc2vec.dv))
type(model_doc2vec.dv)

5007


gensim.models.keyedvectors.KeyedVectors

In [103]:
#set_tags = list(model_doc2vec.docvecs.doctags)
set_tags = list([t.tags[0] for t in tagged_docs])
nb_docs_small = len(set_tags)
print(nb_docs_small)

5007


On récupère le tableau des plongements pour le sauvegarder.

In [104]:
doc_vec_doc2vec = np.zeros(shape=(nb_docs_small, dim_d2v))

i = 0
for t in set_tags:    
    doc_vec_doc2vec[i] = model_doc2vec.dv[t]
    i += 1

doc_vec_doc2vec.shape

(5007, 10)

In [105]:
doc_ids_small = [int(t) for t in set_tags]
nbw_docs_small = [nbw_docs[i-1] for i in doc_ids_small]

col_p = np.array(nbw_docs_small).reshape(nb_docs_small,1)
col_ids = np.array(doc_ids_small).reshape(nb_docs_small,1)
data_to_save = np.hstack([doc_vec_doc2vec, col_p, col_ids])
np.savetxt('vec_doc_doc2vec.csv', data_to_save, delimiter='\t')

De nombreuses autres méthodes existent pour construire des représentations de documents, par exemple :

* InferSent (EMNLP 2017)
* Universal Sentence Encoder (EMNLP 2018)
* SentenceBERT (EMNLP 2019)

N'hésitez pas à consulter la page suivante qui décrit ses approches et comment les implémenter :

https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/

## Clustering de documents


L'objectif est de vous montrer comment utiliser un algorithme simple de clustering (ici, k-means). Bien sûr, l'intérêt d'utiliser un espace vectoriel est de pouvoir utiliser de nombreux autres algorithmes, comme des modèles de mélange, etc.

In [106]:
doc_vec.shape

(8608, 300)

In [118]:
from sklearn.cluster import KMeans

k = 10

km_10 = KMeans(n_clusters=k, random_state=0).fit(doc_vec)

In [119]:
import pandas
pandas.Series(km_10.labels_).value_counts()

8    2463
4    2152
7     918
3     892
5     769
6     398
1     364
2     288
9     276
0      88
dtype: int64

In [120]:
ndocs

8608

In [121]:
col_p = np.array(nbw_docs).reshape(ndocs,1)
col_ids = np.arange(1, ndocs+1).reshape(ndocs,1)
clu_lab = np.array(km_10.labels_).reshape(ndocs,1)
data_to_save = np.hstack([doc_vec, col_p, col_ids, clu_lab])
np.savetxt('vec_doc_naive_cl10.csv', data_to_save, delimiter='\t')

Idem avec la représentation obtenue à l'aide de doc2vec.

In [122]:
km_10_doc2vec = KMeans(n_clusters=10, random_state=0).fit(doc_vec_doc2vec)

In [123]:
doc_ids_small = [int(t) for t in set_tags]
nbw_docs_small = [nbw_docs[i-1] for i in doc_ids_small]
clu_lab_small = np.array(km_10_doc2vec.labels_).reshape(nb_docs_small,1)

col_p = np.array(nbw_docs_small).reshape(nb_docs_small,1)
col_ids = np.array(doc_ids_small).reshape(nb_docs_small,1)
data_to_save = np.hstack([doc_vec_doc2vec, col_p, col_ids, clu_lab_small])
np.savetxt('vec_doc_doc2vec_clu10.csv', data_to_save, delimiter='\t')

Références :
   
* Le, Quoc, and Tomas Mikolov. Distributed representations of sentences and documents. International Conference on Machine Learning (ICML), 2014.
* Alexis Conneau, Douwe Kiela, Holger Schwenk, Loic Barrault, Antoine Bordes. Supervised Learning of Universal Sentence Representations from Natural Language Inference Data, EMNLP 2017.
* Daniel Cera, Yinfei Yanga, Sheng-yi Konga, Nan Huaa, Nicole Limtiacob, Rhomni St. Johna, Noah Constanta, Mario Guajardo-Ce ́spedesa, Steve Yuanc, Chris Tara, Yun-Hsuan Sunga, Brian Stropea. Universal Sentence Encode for Englishr, EMNLP 2018.
* Nils Reimers, Iryna Gurevych. Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks, EMNLP 2019.


Nous n'aurons pas le temps de le faire en cours, mais on comprend bien qu'il est également immédiat de déployer des algorithmes de classification à partir de ces représentations.